# Borrower Reliability Research


In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Data preprocessing

### Removing gaps

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Handling anomalous values

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Two types (unemployed and pensioners) will get abnormally large values. Correcting such values is difficult, so leave them as they are. Moreover, you will not need this column for research.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Removing gaps (continued)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Changing Data Types

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Duplicate Handling

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Data categorization

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
# list of unique purposes for taking a loan
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Exploring data and answering questions

#### 1. Is there a relationship between the number of children and loan repayment on time?

In [22]:
data_children = data.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum', 'mean', 'count'])
data_children.head(10) 

,sum,mean,count
,debt,debt,debt
children,,,
0,1063,0.075438,14091
1,444,0.092346,4808
2,194,0.094542,2052
3,27,0.081818,330
4,4,0.097561,41
5,0,0.000000,9


**Conclusion:** From these results, we can conclude that clients without children are more responsible for their debt obligations, in contrast to people with five children, who completely neglect their obligations and pay nothing. With regard to clients with three children, it can also be said that they are conscientious about paying on time. Then there are clients with one, two and four children.

#### 2. Is there a relationship between marital status and loan repayment on time?

In [23]:
data_family_status = data.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum', 'mean', 'count'])
data_family_status.head(10) 

,sum,mean,count
,debt,debt,debt
family_status,,,
Не женат / не замужем,273,0.097639,2796
в разводе,84,0.070648,1189
вдовец / вдова,63,0.066246,951
гражданский брак,385,0.093130,4134
женат / замужем,927,0.075606,12261


**Conclusion:** It can be concluded that widowed/widowed clients are the most conscientious payers. Divorced clients are in second place, and clients who are married are almost at the same level. Further, based on the data, we see that clients who are in a civil marriage or not married / not married are the worst performers of their debt obligations.

#### 3. Is there a relationship between income level and loan repayment on time?

In [24]:
data_total_income_category = data.pivot_table(index = ['total_income_category'], values = 'debt', aggfunc = ['sum', 'mean', 'count'])
data_total_income_category.head(10) 

,sum,mean,count
,debt,debt,debt
total_income_category,,,
A,2,0.080000,25
B,354,0.070602,5014
C,1353,0.084982,15921
D,21,0.060172,349
E,2,0.090909,22


**Conclusion:** Speaking about the relationship between income level and loan repayment on time, we can say that people with income from 30,000 to 50,000 (category 'D') are the most responsible. After them come people with a salary from category 'B'. Least of all loans are taken by clients with the highest incomes and those with the lowest.

#### 4. How do different purposes of a loan affect its repayment on time?

In [25]:
data_purpose_category = data.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = ['sum', 'mean', 'count'])
data_purpose_category.head(10) 

,sum,mean,count
,debt,debt,debt
purpose_category,,,
операции с автомобилем,400,0.093480,4279
операции с недвижимостью,780,0.072551,10751
получение образования,369,0.092528,3988
проведение свадьбы,183,0.079118,2313


**Conclusion:** Based on the results of the study, it can be said that clients whose purpose of the loan is real estate transactions are most inclined to repay on time. Almost the same probability is holding a wedding. Worst of all return loans for education and car operations.

#### 5. Give possible reasons for gaps in the original data.

*Answer:* Possible reasons for gaps may be the unwillingness of customers to indicate information in optional paragraphs/clarify information/indicate truthful information, lack of accurate information.
Among other things, technical failures in the collection of information are possible.

Gaps can be technical and human, people most often do not want to fill in optional items. Lack of precise information, such as the exact number of days since starting a job.

#### 6. Explain why filling in the gaps with the median is the best solution for scale variables.

*Answer:* If there is no relationship between variables, the most efficient way is to use median values, since the median value provides more objective information than the average value. When removing gaps, we can miss some important information, so sometimes you need to resort to the median.

If there is no relationship between variables, then a simple and often very effective way to fill in the gaps is to use averages. In the event that there is doubt about the normal distribution of data, it is preferable to use the median. The median value gives more objective information.

### General conclusion

In [37]:
def age(dob_years):
    if dob_years <=30:
        return 'молодые'
    if dob_years >=31 and dob_years <=55:
        return 'взрослые'
    if dob_years >=56:
        return 'взрослые'
    
data['age'] = data['dob_years'].apply(age)

In [38]:
data_children = data.pivot_table(index = ['children', 'age'], values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_children.head(20)

count  sum      mean
                    debt debt      debt
children age                           
0        взрослые  11875  824  0.069389
         молодые    2216  239  0.107852
1        взрослые   3737  327  0.087503
         молодые    1071  117  0.109244
2        взрослые   1598  143  0.089487
         молодые     454   51  0.112335
3        взрослые    278   24  0.086331
         молодые      52    3  0.057692
4        взрослые     35    3  0.085714
         молодые       6    1  0.166667
5        взрослые      9    0  0.000000

In [39]:
data_family = data.pivot_table(index = ['family_status', 'age'], values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_family.head(20)

count  sum      mean
                                 debt debt      debt
family_status         age                           
Не женат / не замужем взрослые   1769  158  0.089316
                      молодые    1027  115  0.111977
в разводе             взрослые   1064   67  0.062970
                      молодые     125   17  0.136000
вдовец / вдова        взрослые    939   61  0.064963
                      молодые      12    2  0.166667
гражданский брак      взрослые   3331  299  0.089763
                      молодые     803   86  0.107098
женат / замужем       взрослые  10429  736  0.070572
                      молодые    1832  191  0.104258

In [40]:
data_income = data.pivot_table(index = ['total_income_category', 'age'], values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_income.head(20)

count   sum      mean
                                 debt  debt      debt
total_income_category age                            
A                     взрослые     25     2  0.080000
B                     взрослые   4203   284  0.067571
                      молодые     811    70  0.086313
C                     взрослые  12975  1015  0.078227
                      молодые    2946   338  0.114732
D                     взрослые    308    18  0.058442
                      молодые      41     3  0.073171
E                     взрослые     21     2  0.095238
                      молодые       1     0  0.000000

In [41]:
data_purpose = data.pivot_table(index = ['purpose_category', 'age'], values = 'debt', aggfunc = ['count', 'sum', 'mean'])
data_purpose.head(20)

count  sum      mean
                                   debt debt      debt
purpose_category         age                          
операции с автомобилем   взрослые  3576  313  0.087528
                         молодые    703   87  0.123755
операции с недвижимостью взрослые  8792  589  0.066993
                         молодые   1959  191  0.097499
получение образования    взрослые  3255  276  0.084793
                         молодые    733   93  0.126876
проведение свадьбы       взрослые  1909  143  0.074908
                         молодые    404   40  0.099010

As a result, if you add new data, you can make the following recommendations to the customer:
An excellent client is considered: An adult family (age 31+) with 5 children. A bad client is: Young family (under 30 years old) with (4+) - children.
A great client is: A divorced adult (age 31+). A bad client is: Young people who are divorced (under the age of 30).
A great customer is: A low-income adult (age 31 or older). A bad client is: Young people with an upper-middle income (under 30 years of age).
An excellent client is considered to be: An adult with the purpose of buying a property (age 31 or older). A bad client is: Young people with the purpose of education (age under 30).